# Sequence 2 Sequence
#### Reference:
* https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

In [2]:
from torch import optim

import torch
import torch.nn as nn
import torch.nn.functional as f

# activate cuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
# beginning token and ending token of sentence
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {
            0: "SOS",
            1: "EOS"
        }
        self.n_words = 2 # only SOS & EOS at this moment
    
    def addSentence(self, sentence, language):
        if language == "EN":
            for word in sentence.split(' '):
                self.addWord(word)
        elif language == "ZH":
            for word in sentence:
                self.addWord(word)
                
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[slef.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

John Appleseed
